# Imports

In [1]:
import os
import math
import json
import collections

import pandas as pd
import numpy as np

from src.pre_processing.macros import column_groups, DATA_PATH, DATA_SPLIT_PATH, ORIGINAL_DATASET_NAME, DATA_PREPROC_PATH

# Load Original

In [76]:
df = pd.read_csv(os.path.join(DATA_PATH, ORIGINAL_DATASET_NAME), low_memory=False)
df = df.set_index("id_student")
df

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19,student_questionnaire,principals_questionnaire,family_questionnaire,...,p331a,p331b,p331c,p331d,p331e,p331f,p331g,p331j,pfc,rep
id_student,,,,,,,,,,,,,,,,,,,,,
1,33613.0,2016,3,A,2415.0,14374.0,569.0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19294.0,2016,3,A,1842.0,8001.0,273.0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19587.0,2016,3,NaN,1432.0,8142.0,82.0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29985.0,2016,3,A,2280.0,12800.0,505.0,1,1,1,...,2.0,2.0,3.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN
5,6982.0,2016,3,A,2040.0,2606.0,390.0,1,1,1,...,4.0,NaN,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,2500.0,2019,6,B,1278.0,NaN,271.0,1,1,0,...,4.0,4.0,4.0,4.0,4.0,NaN,5.0,3.0,10.0,2.0
83854,5784.0,2019,6,A,1079.0,NaN,78.0,1,1,1,...,3.0,4.0,4.0,4.0,4.0,NaN,5.0,4.0,3.0,2.0
83855,7708.0,2019,6,X,1486.0,3036.0,459.0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# Import ranges of each feature
with open(os.path.join(DATA_PATH, "meta_data_original.json")) as file:
    meta_data = json.load(file)

In [116]:
# Check each features is in range
for column_group in meta_data["groups"]:
    for col in column_group["columns"]:
        value = column_group["values"][0]
        if value != ["Unknown"]:
            if type(value) == dict:
                col_min = df[col].min()
                col_max = df[col].max()
                if col_min < int(value["min"]) or col_max > int(value["max"]):
                    print(f""""NO IN LIMIT\n\tcol:{col}, col_min:{col_min}, col_max:{col_max}, min_value:{value["min"]}, max_value:{value["max"]} """)
            elif type(column_group["values"][0]) == list:
                distinct_in_col = [elem for elem in df[col].unique() if type(elem) == str or not math.isnan(elem)]
                value = value if not value[0].isdigit() else [int(elem) for elem in value]
                # if distinct_in_col not in value:
                if not set(distinct_in_col).issubset(set(value)):
                    print(f""""NO IN LIST\n\tcol:{col}, distinct_in_col:{distinct_in_col}, value:{value}""")


"NO IN LIMIT
	col:ESCS, col_min:-2.706281, col_max:3.293104, min_value:1, max_value:25 
"NO IN LIST
	col:p331g, distinct_in_col:[1.0, 3.0, 4.0, 5.0, 2.0], value:[1, 2, 3, 4]
"NO IN LIST
	col:p331j, distinct_in_col:[4.0, 3.0, 5.0, 2.0, 1.0], value:[1, 2, 3, 4]


In [55]:
import 
collections.Counter(df["a1"].unique().tolist()[:2]) == collections.Counter(["2", "1"])

False

[2.0, 1.0]

# Split and Save

In [2]:
# identifiers pre-processing, i.e., change float columns to int
int_identifiers = [col for col in column_groups["identifiers"] if col not in ["id_class_group"]]
df[int_identifiers] = df[int_identifiers].astype('Int64')
df

NameError: name 'df' is not defined

In [5]:
for group, columns in column_groups.items():
    print(group, len(columns))

identifiers 7
info 6
student_scores 6
student_questionnaire 154
school_principals_questionnaire 138
family_questionnaire 91
teacher_questionnaire 158


In [6]:
for group, columns in column_groups.items():
    df[columns].to_csv(os.path.join(DATA_SPLIT_PATH, f"{group}.csv"))

# Load Pre-processed

In [15]:
df = pd.read_csv(os.path.join(DATA_PREPROC_PATH, "merged.csv"), low_memory=False)
df = df.set_index("id_student")
print(df.shape)
df

(80174, 147)


,s_gender,s_birth_year,s_is_living_with_mother,s_is_living_with_father,s_is_living_with_siblings,s_is_living_with_other_relatives,s_has_repeated,s_frequency_of_skips,s_homeworks_day_per_week,s_extent_of_teachers_affinity,...,t_extent_of_student_involvement_during_class,t_extent_of_teaching_methods_variety,t_extent_of_opinion_on_school,t_extent_of_class_behaviour,t_extent_of_resource_variety,t_extent_of_good_work_by_non_teachers,t_number_of_individual_training_topics,t_number_of_subjects_taught,t_extent_of_satisfaction_job_and_school,t_extent_of_results_satisfaction
id_student,,,,,,,,,,,,,,,,,,,,,
1,FEMALE,2007.0,True,True,NaN,True,False,4.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MALE,2007.0,True,True,True,NaN,True,4.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FEMALE,2007.0,True,True,True,NaN,False,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MALE,2007.0,True,True,True,NaN,False,4.0,3.0,4.0,...,2.666667,3.0,4.0,2.5,2.375,2.666667,NaN,NaN,NaN,3.0
5,FEMALE,2007.0,True,True,True,NaN,False,4.0,1.0,4.0,...,3.000000,3.2,4.0,3.5,2.875,3.666667,1.0,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,FEMALE,NaN,NaN,NaN,NaN,NaN,True,4.0,2.0,4.0,...,2.666667,3.4,4.0,2.5,2.250,4.000000,1.0,NaN,4.000000,1.5
83854,FEMALE,NaN,NaN,NaN,NaN,NaN,False,4.0,4.0,2.0,...,3.000000,3.2,3.0,3.0,2.500,4.000000,1.0,NaN,3.166667,2.5
83855,MALE,NaN,NaN,NaN,NaN,NaN,False,4.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column]  / df_scaled[column].abs().max()
    return df_scaled

In [4]:
extent_of_cols = [col for col in df if col.startswith('extent_of')]
frequency_of_cols = [col for col in df if col.startswith('frequency_of')]
# agreement_of_cols = [col for col in df if col.startswith('agreement_of')]


In [81]:
for col in df.columns:
    if "gender" in col:
        print(col)

s_gender
p_gender
p_group_criteria_gender
t_gender


In [20]:
# Import meta data of merged
with open(os.path.join(DATA_PATH, "meta_data_new_py.json")) as file:
    meta_data = json.load(file)

In [116]:
# Write
# with open(os.path.join(DATA_PATH, "meta_data_new_py.json"), 'w') as f:
#     f.write(json.dumps(meta_data))

In [41]:
# for cmn, details in meta_data.items():
#     value = details["values"][0][0]
#     if value == ["1", "2", "3", "4"]:
#         value = {"min": 1, "max": 4}
#         print(value)

In [14]:
df["f_monthly_household_income"]

id_student
1                   NaN
2                   NaN
3                   NaN
4                   NaN
5                   NaN
              ...      
83853               NaN
83854    MORE_THAN_3500
83855        UP_TO_2000
83856         NO_ANSWER
83857        UP_TO_1000
Name: f_monthly_household_income, Length: 80174, dtype: object

In [10]:
categories = ["s", "p", "f", "t"]
attrs = ["extent_of", "frequency_of"]

for col, details in meta_data.items():
    if any(col.startswith(s) for s in [f"{c}_{a}" for c in categories for a in attrs]):
        value = details["values"][0][0]
        if value == ["1", "2", "3"]:
            value = {"min": 1, "max": 3}
        if value == ["1", "2", "3", "4"]:
            value = {"min": 1, "max": 4}
        if value == ["1", "2", "3", "4", "5"]:
            value = {"min": 1, "max": 5}
        # details["values"][0][0] = value
        print(col, details["values"][0][0])

s_frequency_of_skips {'min': 1, 'max': 4}
s_frequency_of_computer_usage {'min': 1, 'max': 4}
s_frequency_of_internet_usage {'min': 1, 'max': 4}
s_frequency_of_work_with_teachers {'min': 1, 'max': 4}
s_frequency_of_materials_in_class {'min': 1, 'max': 4}
s_frequency_of_evaluations {'min': 1, 'max': 4}
s_extent_of_teachers_affinity {'min': 1, 'max': 4}
s_extent_of_classmates_affinity {'min': 1, 'max': 4}
s_extent_of_teacher_performance {'min': 1, 'max': 4}
s_extent_of_class_env {'min': 1, 'max': 4}
s_extent_of_school_satisfaction {'min': 1, 'max': 4}
s_extent_of_math_affinity {'min': 1, 'max': 4}
s_extent_of_reading_affinity {'min': 1, 'max': 4}
p_extent_of_factors_limiting_effectiveness {'min': 1, 'max': 4}
p_extent_of_inconveniences {'min': 1, 'max': 4}
p_extent_of_problems {'min': 1, 'max': 4}
p_extent_of_management {'min': 1, 'max': 4}
p_extent_of_satisfaction {'min': 1, 'max': 4}
p_extent_of_agreement {'min': 1, 'max': 4}
p_extent_of_satisfaction_students_results {'min': 1, 'max': 4

In [22]:
# Check each features is in range
for col, details in meta_data.items():
    # print(col)
    value = details["values"][0][0]
    # if value == ["1", "2", "3", "4"]:
    #     value = {"min": 1, "max": 4}
    if value not in [["Unknown"], ["Integer"], ["Float"]]:
        if type(value) == dict:
            try:
                col_min = df[col].min()
                col_max = df[col].max()
                if col_min < int(value["min"]) or col_max > int(value["max"]):
                    print(f""""NO IN LIMIT\n\tcol:{col}, col_min:{col_min}, col_max:{col_max}, min_value:{value["min"]}, max_value:{value["max"]} """)
            except:
                print([elem for elem in df[col].unique() if type(elem) == str or not math.isnan(elem)])
        elif type(value) == list:
            distinct_in_col = [elem for elem in df[col].unique() if type(elem) == str or not math.isnan(elem)]
            value = value if not value[0].isdigit() else [int(elem) for elem in value]
            # if distinct_in_col not in value:
            if not set(distinct_in_col).issubset(set(value)):
                print(f""""NO IN LIST\n\tcol:{col}, distinct_in_col:{distinct_in_col}, value:{value}""")


"NO IN LIST
	col:s_is_living_with_mother, distinct_in_col:[True, False], value:['False', 'True']
"NO IN LIST
	col:s_is_living_with_father, distinct_in_col:[True, False], value:['False', 'True']
"NO IN LIST
	col:s_is_living_with_siblings, distinct_in_col:[True], value:['False', 'True']
"NO IN LIST
	col:s_is_living_with_other_relatives, distinct_in_col:[True], value:['False', 'True']
"NO IN LIST
	col:s_has_repeated, distinct_in_col:[False, True], value:['False', 'True']
[196.0, 157.0, 62.0, 131.0, 118.0, 123.0, 202.0, 289.0, 260.0, 85.0, 317.0, 132.0, 252.0, 143.0, 94.0, 152.0, 63.0, 214.0, 168.0, 106.0, 293.0, 115.0, 82.0, 282.0, 100.0, 170.0, 388.0, 287.0, 660.0, 875.0, 175.0, 220.0, 464.0, 125.0, 364.0, 199.0, 0.0, 146.0, 268.0, 284.0, 120.0, 147.0, 312.0, 408.0, 372.0, 56.0, 209.0, 410.0, 322.0, 130.0, 188.0, 213.0, 84.0, 54.0, 72.0, 249.0, 362.0, 230.0, 280.0, 292.0, 244.0, 80.0, 57.0, 286.0, 310.0, 184.0, 128.0, 50.0, 121.0, 204.0, 205.0, 236.0, 290.0, 29.0, 36.0, 124.0, 110.0, 126